<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background- padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Analyzing Proximity to Climate Risk At Scale using Vantage and ClearScape Analytics
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>The effects of climate change pose risks to persons and property in the proximity of the more vulnerable geographic regions on the planet.  The ability to rapidly analyze individuals in the path of catastrophic weather or other types of events can save countless lives.  Additionally, having this capability combined with other enterprise planning and analytics processes can assist with business operations, disaster preparedness, and long-term strategic planning.</p>

<p style = 'font-size:16px;font-family:Arial'>Teradata <b style = 'color:#00b2b1'>ClearScape Analytics Functions</b> combine traditional analytics, machine learning, and advanced statistical processing along with <b style = 'color:#00b2b1'>Geospatial</b> capabilities.  This allows enterprises to rapidly analyze this geographic-related information in real-time at any scale - effectively understanding the impacts of these events on entire populations.</p>

<p style = 'font-size:18px;font-family:Arial'><b>Demo Overview</b></p>        
<p style = 'font-size:16px;font-family:Arial'>In 2022, the North Island of New Zealand experienced drastic and catastrophic flooding.  This demonstration notebook will illustrate how analysts can leverage location and proximity information at scale to analyze which specific addresses are within a certain proximity to flooding.  This information can be used for lifesaving flood risk warnings, better disaster preparedness, or even more efficient insurance and planning regulations.<p>
    
<p style = 'font-size:16px;font-family:Arial'>The steps are as follow:</p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li><b style = 'color:#00b2b1'>Inspect Geospatial data</b> using client-side tools and visualization</li>
    <li><b style = 'color:#00b2b1'>Analyze Proximity at Scale</b> using ClearScape Analytic functions to calculate flood location centroids and risk boundaries, and then apply the analysis at scale to analyze every New Zealand address and asses high, medium, and low risk based on proximity to the flood zones</li>
    </ol>

<p style = 'font-size:16px;font-family:Arial'>For the demo we have taken approximately 60k address sample from 1.6million addresses of North Island.

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>1. Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial'>Import the libraries needed.</p>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import getpass
import geopandas as gpd
import folium
from branca.element import Figure
import matplotlib.pyplot as plt
import pandas as pd
from shapely import wkt
from folium import plugins

from teradataml import *

from IPython.display import display as ipydisplay

display.suppress_vantage_runtime_warnings = True

display.max_rows=5

<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, then <b>use down arrow</b> to go to next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql("SET query_band='DEMO=Flood_Proximity_Analysis_Python.ipynb;' UPDATE FOR SESSION;")

<p style = 'font-size:18px;font-family:Arial'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage. There are two statements in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_NZFloods_cloud');"
 # takes about 30 seconds, estimated space: 0 MB
%run -i ../run_procedure.py "call get_data('DEMO_NZFloods_local');" 
# takes about 30 seconds, estimated space: 3 MB

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>2. Inspect the Data</b></p>
<p style = 'font-size:16px;font-family:Arial'>Data was obtained in ESRI shape format, and loaded into Vantage using teradataml client libraries.</p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li><b style = 'color:#00b2b1'>New Zealand Street Addresses</b> Addresses from North Island</li>
    <li><b style = 'color:#00b2b1'>Flood Areas</b>  Seven areas in total</li>
    </ul>

<p style = 'font-size:16px;font-family:Arial'>Analyze some of the data locally:</p>
            <ul style = 'font-size:16px;font-family:Arial'>
                <li><b>Inspect the Data</b> Look at a sample of rows, row counts</li>
                <li><b>Map the Flood Zones</b> Using client-side tools</li>
                <li><b>Calculate and Map Centroids</b> Use client tools to calculate the centroids of each region</li>
            </ul>


<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>2.1 Street Addresses</b></p>

In [ ]:
tdf_addrs = DataFrame('"DEMO_NZFloods"."nz_addrs_geoV"')

In [ ]:
print('NZ Street Addresses with Lat, Lon, and POINT object:')
ipydisplay(tdf_addrs)
print('Sample 60k Addresses:')
ipydisplay(tdf_addrs.shape)

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>2.1 Flood Risk Areas</b></p>

In [ ]:
tdf_flood = DataFrame('"DEMO_NZFloods"."nz_floods_geoV"')

#df_flood.reset_index(inplace = True)
tdf_flood.to_pandas()

In [ ]:
tdf_flood.shape

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>2.3 Putting the Flood Zones on a Map</b></p>

<p style = 'font-size:16px;font-family:Arial'>This demonstration uses the <a href = 'https://python-visualization.github.io/folium/'>folium</a> package along with <a href = 'https://geopandas.org/en/stable/'>geopandas</a> to overlay the flood zone data on an interactive map. We will start by showing the basic map (named "m").</p>

In [ ]:
fig = Figure(width=600, height=400)
m = folium.Map(location=[-38.6661126,175.9408615], zoom_start=7)
minimap = plugins.MiniMap()
m.add_child(minimap)


<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>2.4 Draw the Flood Regions</b></p>

<p style = 'font-size:16px;font-family:Arial'>Use Pandas, GeoPandas, and Folium to paint the flood zones on top of the basic map (named "m")</p>

In [ ]:
# Load the flood zone data into local geopandas dataframe

df = tdf_flood.to_pandas()
df["GEOM"] = df["GEOM"].apply(wkt.loads)
df_flood = gpd.GeoDataFrame(df, geometry="GEOM", crs=4326)


# add the flood zones to the map
for _, r in df_flood.iterrows():
    sim_geo = gpd.GeoSeries(r['GEOM']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 0.5})
    geo_j.add_to(m)
m

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>2.5 Calculate Centroids</b></p>

<p style = 'font-size:16px;font-family:Arial'>A centroid is the geometric centre of the object or shape. In order to properly compute geometric properties, in this case centroids, of the geometries, we need to project the data to a projected coordinate system.</p>


In [ ]:
# Project to New Zealand Transverse Mercator projected Coordinate Reference System

df_flood = df_flood.to_crs(epsg=2193)

# Access the centroid attribute of each polygon
df_flood['centroid'] = df_flood.centroid

# Project to WGS84 geographic crs
# geometry (active) column
# EPSG 4326 is the projection based on WGS84 (GPS) coordinate system
df_flood = df_flood.to_crs(epsg=4326)

# Centroid column
df_flood['centroid'] = df_flood['centroid'].to_crs(epsg=4326)

df_flood.head()

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>2.6 Put Centroids on the Map</b></p>

In [ ]:
for _, r in df_flood.iterrows():
    lat = r['centroid'].y
    lon = r['centroid'].x
    folium.Marker(location=[lat, lon], popup = 'FID: {}'.format(r['fid'])
                  ).add_to(m)

m

<p style = 'font-size:16px;font-family:Arial'>In the map above, the blue markers are the centroid of each of the flood zones (1-7).

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>3. Geospatial Analysis for Proximity</b></p>
<p style = 'font-size:16px;font-family:Arial'>As we have seen above one can use common client-side tools/python libraries for Geospatial visualization and analysis.  In reality, the true value in this analysis occurs both at the <b>scale</b> of human populations, or <b>combined</b> with other analytic and machine learning insights.  <b style = 'color:#00b2b1'>Teradata Vantage ClearScape Analytics</b> allows just this. Below we can see how we can:</p>
            <ul style = 'font-size:16px;font-family:Arial'>
                <li><b>Calculate Centroids and Buffer Zones</b> around the Flood Zone shapes</li>
                <li><b>Map the Buffer Zones</b> Using client-side tools</li>
                <li><b>Calculate Address Proximity</b> Analyze all 60k Records in-Database in seconds</li>
                <li><b>Map Degree of Risk</b> Based on Proximity calculations</li>
            </ul>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.1 Calculate Centroids and Buffer Zones in-Database</b></p>

<p style = 'font-size:16px;font-family:Arial'>Use the <a href = ''>centroid</a> property as well as the <a href = 'https://docs.teradata.com/r/Lake/Teradata-Package-for-Python-Function-Reference-on-VantageCloud-Lake/Geospatial/teradataml-GeoDataFrame/Methods/Geospatial-Specific-Methods/Methods-for-All-Types-of-Geometries/buffer'>buffer</a> method of the teradataml DataFrame to inspect and calculate this data in-database.  Note this python syntax is automatically translated to SQL to run in the Vantage system.  The <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Geospatial-Data-Types/July-2021/ST_Geometry-Constructors-and-Methods/ST_Centroid-Method'>ST_Centroid</a> and <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Geospatial-Data-Types/July-2021/ST_Geometry-Constructors-and-Methods/ST_Buffer-Method'>ST_Buffer</a> SQL functions can be run directly as well to perform the same calculations.</p>
<p style = 'font-size:16px;font-family:Arial'>For the following steps, we will focus on Zone 7 located in the southeast side of North Island around Hastings.</p>

In [ ]:
# Get a reference to the data in Vantage
tdf_flood = GeoDataFrame('"DEMO_NZFloods"."nz_floods_geoV"')

# Call the teradataml assign() method to create two new columns "CNTRD" and "CNTRD_BUFFER" 
# containing the centroid and .5KM boundary shape respectively

tdf_flood = tdf_flood.assign(CNTRD = tdf_flood.GEOM.centroid, CNTRD_BUFFER = tdf_flood.GEOM.centroid.buffer(.5))

# Inspect the new columns for Flood Zone 7
tdf_flood[tdf_flood['fid'] == 7].to_pandas()

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.2 Map the Buffer</b></p>

<p style = 'font-size:16px;font-family:Arial'>Retrieve the buffer shape to the client, and use GeoPandas methods to simplify the shape so zone 7 can be easily drawn on the map.</p>

In [ ]:
df_buffer = tdf_flood[tdf_flood['fid'] == 7].to_pandas()
df_buffer["CNTRD_BUFFER"] = df_buffer["CNTRD_BUFFER"].apply(wkt.loads)
gdf_buffer = gpd.GeoDataFrame(df_buffer, geometry="CNTRD_BUFFER", crs=4326)
gdf_buffer

In [ ]:
for _, r in gdf_buffer.iterrows():
    sim_geo = gpd.GeoSeries(r['CNTRD_BUFFER']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'grey', 'color': 'grey', 'weight': 0.5})
#     folium.Popup(r['ZONETYPE']).add_to(geo_j)
    geo_j.add_to(m)
m

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.3 Find all addresses within the Buffer</b></p>

<p style = 'font-size:16px;font-family:Arial'>First, join the buffer shape to the address information.  Next, use the <a href = 'https://docs.teradata.com/r/Lake/Teradata-Package-for-Python-Function-Reference-on-VantageCloud-Lake/Geospatial/teradataml-GeoDataFrame/Methods/Geospatial-Specific-Methods/Methods-for-All-Types-of-Geometries/within'>within</a> method to find all addresses inside the buffer boundary.</p>

In [ ]:
tdf_nz_addrs = GeoDataFrame('"DEMO_NZFloods"."nz_addrs_geoV"')
tdf_nz_addrs = tdf_nz_addrs.join(tdf_flood[tdf_flood['fid'] == 7][['CNTRD_BUFFER']], how = 'cross')


tdf_nz_addrs[tdf_nz_addrs['POINT'].within(tdf_nz_addrs['CNTRD_BUFFER']) == 1].to_sql(table_name = 'addrs_in_buffer', 
                                                                                     if_exists = 'replace')
tdf_within = GeoDataFrame('addrs_in_buffer')
print('Number of Addresses within Flood Zone Seven Buffer:')
ipydisplay(tdf_within.shape)

print('Sample of the Data:')
ipydisplay(tdf_within[['address_id', 'full_add_2','POINT']].to_pandas(num_rows = 2))

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.4 Calculate distance in KMs to all addresses within the buffer from Flood Zone 7</b></p>

<p style = 'font-size:16px;font-family:Arial'>This step requires two Teradata Geospatial methods;</p>
<ol style = 'font-size:16px;font-family:Arial'>
<li><b><a href = 'https://docs.teradata.com/r/Lake/Teradata-Package-for-Python-Function-Reference-on-VantageCloud-Lake/Geospatial/teradataml-GeoDataFrame/Methods/Geospatial-Specific-Methods/Methods-for-All-Types-of-Geometries/transform'>transform</a></b> Which will transform our geometry objects to the desired coordinate reference system.  For the SQL syntax see <a href = 'https://docs.teradata.com/r/W1AEeHO2cxTi3Sn7dtj8hg/Tu9pmmM_0OQEqwK4eq86fg'>ST_Transform</a>.  This function takes two primary arguments in "Well Known Text" (WKT) Format; the <b>target</b> Coordinate System, and the <b>source</b> coordinate system.  Teradata stores all supported CRS in a specific database table "SYSSPATIAL"."SPATIAL_REF_SYS".  The values from this table can be used as the required arguments.</li>
    <li><b><a href = 'https://docs.teradata.com/r/Lake/Teradata-Package-for-Python-Function-Reference-on-VantageCloud-Lake/Geospatial/teradataml-GeoDataFrame/Methods/Geospatial-Specific-Methods/Methods-for-All-Types-of-Geometries/distance'>distance</a></b> which will calculate the distance between two geometry objects.  For the SQL syntax see <a href = 'https://docs.teradata.com/r/W1AEeHO2cxTi3Sn7dtj8hg/V30FcBuekQCTfWVD~ifJag'>ST_Distance</a>.
        </ol>

<p style = 'font-size:16px;font-family:Arial'>Once we have the distance of each selected address to the actual flood boundary, filter the results to return only addresses less than 10KM from the flood zone.</p>

<p style = 'font-size:16px;font-family:Arial'>This set step is a good example of where SQL can be easier to express than the associated teradataml method.  Note here we can combine the various tables including the CRS, as well as distance filtering and the original flood zones table data.</p>

In [ ]:
qry = '''
SELECT 
    AD.TOWN_CITY,
    AD.FULL_ADD_2,
    AD.POINT,
    FZ.GEOM.ST_TRANSFORM(SRS_1.srtext,SRS_2.srtext).ST_DISTANCE(AD.POINT.ST_TRANSFORM(SRS_1.srtext,SRS_2.srtext))*1/1000 as DIST_KM
FROM 
    addrs_in_buffer AD,
    DEMO_NZFloods.nz_floods_geoV FZ,
    sysspatial.SPATIAL_REF_SYS SRS_1,
    sysspatial.SPATIAL_REF_SYS SRS_2
WHERE
    FZ.FID = 7
    AND SRS_1.AUTH_SRID = 2193 -- NZTM (to) Projected System
    AND SRS_2.AUTH_SRID = 4326 -- WGS84 (from) Geographic System
    AND FZ.GEOM.ST_TRANSFORM(SRS_1.srtext,SRS_2.srtext).ST_DISTANCE(AD.POINT.ST_TRANSFORM(SRS_1.srtext,SRS_2.srtext))*1/1000 < 10;
'''

tdf_st_dist = DataFrame.from_query(qry)
print("Number of Addresses within 10KM of the Flood Zone:")
ipydisplay(tdf_st_dist.shape)

In [ ]:
df_st_dist = tdf_st_dist.sample(frac = .1).to_pandas(all_rows = True)
df_st_dist["POINT"] = df_st_dist["POINT"].apply(wkt.loads)
gdf_st_dist = gpd.GeoDataFrame(df_st_dist, geometry="POINT", crs=4326)
gdf_st_dist

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.5 Use Client-side tools to create risk bands</b></p>

<p style = 'font-size:16px;font-family:Arial'>Using GeoPandas, paint each address as follows</p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li><b style = 'color:red'>Red</b> for addresses inside the Flood Zone</li>
    <li><b style = 'color:#8B8000'>Yellow</b> for addresses 0 to 5 KM away</li>
    <li><b style = 'color:green'>Green</b> for addresses 5 to 10 KM away</li>
    </ul>
<p style = 'font-size:16px;font-family:Arial'>Note there are about 400 addresses inside the 10 KM boundary to Flood Zone 7.</p>

In [ ]:
# Create circle markers for each address
# Color them based on the distance calculation above

for _, r in df_st_dist[df_st_dist.DIST_KM == 0].iterrows():
    lat = r['POINT'].y
    lon = r['POINT'].x
    folium.CircleMarker(location=[lat, lon], 
                        popup='ADDR: {} <br> DIST: {}'.format(r['full_add_2'], r['DIST_KM']), 
                        radius=1,
                        color = 'red').add_to(m)

for _, r in df_st_dist[df_st_dist['DIST_KM'].between(0.00001, 5)].iterrows():
    lat = r['POINT'].y
    lon = r['POINT'].x
    folium.CircleMarker(location=[lat, lon], 
                        popup='ADDR: {} <br> DIST: {}'.format(r['full_add_2'], r['DIST_KM']), 
                        radius=1,
                        color = 'yellow').add_to(m)
for _, r in df_st_dist[df_st_dist['DIST_KM'].between(5, 10)].iterrows():
    lat = r['POINT'].y
    lon = r['POINT'].x
    folium.CircleMarker(location=[lat, lon], 
                        popup='ADDR: {} <br> DIST: {}'.format(r['full_add_2'], r['DIST_KM']), 
                        radius=1,
                        color = 'green').add_to(m)

m

<p style = 'font-size:18px;font-family:Arial'>Optional - Save map as .html file to view later or share</p>

In [ ]:
m.save("linz_flood_risk_zone7_points.html")

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.6 Heatmap Visualization</b></p>

<p style = 'font-size:16px;font-family:Arial'>Using GeoPandas, get latitude and longitude and create Heat Map interactive map.</p>

In [ ]:
gdf_st_dist = gpd.GeoDataFrame(df_st_dist, geometry="POINT", crs=4326)
gdf_st_dist['lat'] = gdf_st_dist.geometry.y
gdf_st_dist['lon'] = gdf_st_dist.geometry.x

<p style = 'font-size:16px;font-family:Arial'><b>Population in buffer area within 10 KMs radius</b></p>

In [ ]:
fig = Figure(width=850, height=500)
hm = folium.Map(location=[-39.646172303816414, 176.775454544966948], zoom_start=8)


minimap = plugins.MiniMap()
hm.add_child(minimap)

for _, r in df_flood.iterrows():
    sim_geo = gpd.GeoSeries(r['GEOM']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'weight': 0.5})
#     folium.Popup(r['FID']).add_to(geo_j)
    geo_j.add_to(hm)

locations=list(zip(gdf_st_dist.full_add_2, gdf_st_dist.DIST_KM))
popups = ['ADDR: {} <br> DIST: {}'.format(full_add_2, DIST_KM) for (full_add_2, DIST_KM) in locations]

location=gdf_st_dist[['lat','lon']]
plugins.MarkerCluster(location, popups=popups).add_to(hm)
# fig.add_child(hm)
hm

<p style = 'font-size:16px;font-family:Arial'>The above map shows are the total count of the addresses. You can zoom the map to individual address location, click on the aggregated count and the map will zoom in or you can click the '+' button.

<p style = 'font-size:16px;font-family:Arial'><b>Assign risk score in range of 0-100 for a Heat Map gradient value.</b></p>

In [ ]:
gdf_st_dist.loc[gdf_st_dist['DIST_KM'] ==0, 'RISK'] = 100
gdf_st_dist.loc[gdf_st_dist['DIST_KM'].between(0.00001, 5), 'RISK'] = 50
gdf_st_dist.loc[gdf_st_dist['DIST_KM'].between(5, 10), 'RISK'] = 25

heat_data=list(zip(gdf_st_dist['lat'], gdf_st_dist['lon'], gdf_st_dist.RISK))
folium.plugins.HeatMap(heat_data,
                        gradient={0.1: 'blue', 0.3: 'lime', 0.5: 'yellow', 0.7: 'orange', 1: 'red'}, 
                        min_opacity=0.05, 
                        max_opacity=0.9, 
                        radius=25,
                        use_local_extrema=False
                      ).add_to(hm)

folium.LayerControl().add_to(hm)
hm

<p style = 'font-size:16px;font-family:Arial'>The heat map shows the number of properties at risk within an area. As you move your mouse over the circles and zoom in and out, you will see the shape of the area marked with the circle and the basis of the color.  Hundreds of properties would be red, less than 10 might be green. </p>

In [ ]:
hm.save("linz_flood_risk_zone7_heatmap.html")

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.7 Visualization in Plotly</b></p>

<p style = 'font-size:16px;font-family:Arial'>Plotly is another popular visualization tool. Overlay the risk bands on the Plotly street map.</p>

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Densitymapbox(lat=gdf_st_dist['lat'], 
                                 lon=gdf_st_dist['lon'],
                                 z=gdf_st_dist['RISK'],
                                 radius=3,
                                 colorscale=[[0.0, 'blue',],[0.3,'green'],[0.5,'yellow'],[0.7,'orange'],[1.0, 'red']],
                                 zmin=0.0,
                                 zmax=1.0,
                                 opacity=0.5                           
                                ))

fig.update_layout(mapbox_style="open-street-map",
                  mapbox_center_lon=176.775454544966948,
                  mapbox_center_lat=-39.646172303816414,
                  mapbox_zoom=8)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<p style = 'font-size:16px;font-family:Arial'>On this representation the density of properties at a given zoom level changes by color alone.  As you zoom in, the dots will go from red to yellow to orange to green.</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>4. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial'><b>Worktables</b> </p>

In [ ]:
db_drop_table('addrs_in_buffer')

<p style = 'font-size:20px;font-family:Arial'> <b>Database and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_NZFloods');" 
#Takes 10 seconds

In [ ]:
remove_context()

<p style = 'font-size:20px;font-family:Arial'><b>Reference Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'> 
<li>Teradata® Geospatial Utilities User Guide: <a href = 'https://docs.teradata.com/r/Teradata-Geospatial-Utilities-User-Guide/June-2022/Teradata-Geospatial-Utilities-Overview/Welcome-to-Teradata-Tools-and-Utilities-Teradata-Geospatial-Utilities-User-Guide'>https://docs.teradata.com/r/Teradata-Geospatial-Utilities-User-Guide/June-2022/Teradata-Geospatial-Utilities-Overview/Welcome-to-Teradata-Tools-and-Utilities-Teradata-Geospatial-Utilities-User-Guide</a></li>
    
</ul>

<footer style="padding-bottom:35px; background:#91A0AB; ">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>